In [1]:
import pandas as pd
import numpy as np
import tarfile
%reload_ext version_information
np.__version__, pd.__version__

('1.16.4', '0.24.2')

In [ ]:
tar = tarfile.open("lfw_funneled.tgz")
#tar.extractall() #gefährlich, entpackt direkt alles
tar.close()

In [2]:
%version_information numpy, pandas

Software versions
Python 3.7.3 64bit [MSC v.1915 64 bit (AMD64)]
IPython 7.6.1
OS Windows 10 10.0.18362 SP0
numpy 1.16.4
pandas 0.24.2
Wed Oct 30 08:29:26 2019 Mitteleuropäische Zeit